In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Preparing the environment

In [ ]:
!pip install openmim
!mim install mmdet

  Created wheel for openmim: filename=openmim-0.1.5-py2.py3-none-any.whl size=42502 sha256=14ae80d31be8e2f8337f28f5d680a96300610a0b8457d415d0394a528be12060
  Stored in directory: /root/.cache/pip/wheels/16/8b/e1/bdebbbc687aa50224a5ce46fe97a040a0c59f92b34bfc750b6
  Created wheel for ordered-set: filename=ordered_set-4.0.2-py2.py3-none-any.whl size=8219 sha256=7880431192dd9ef539f1ca435c98e7584b26f4fe10367ac4d683baca510e4dda
  Stored in directory: /root/.cache/pip/wheels/73/2b/f6/26e9f84153c25050fe7c09e88f8e32a6be3c7034a38c418319
Successfully built openmim ordered-set
installing mmdet from https://github.com/open-mmlab/mmdetection.git.
Cloning into '/tmp/tmp7fpvgrg4/mmdetection'...
remote: Enumerating objects: 22332, done.
remote: Total 22332 (delta 0), reused 0 (delta 0), pack-reused 22332
Receiving objects: 100% (22332/22332), 25.45 MiB | 26.22 MiB/s, done.
Resolving deltas: 100% (15634/15634), done.
Note: checking out '5e246d5e3bc3310b5c625fb57bc03d2338ca39bc'.

You are in 'detached HE

In [ ]:
!unzip 'dataset.zip'

In [ ]:
import time
import matplotlib
import matplotlib.pylab as plt
import mmcv
from mmcv.runner import load_checkpoint
import mmcv.visualization.image as mmcv_image
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

# fix for colab
plt.rcParams["axes.grid"] = False


def imshow(img, win_name='', wait_time=0):
  plt.figure(figsize=(50, 50))
  plt.imshow(img)


In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git

Cloning into 'mmdetection'...
remote: Enumerating objects: 22332, done.
remote: Total 22332 (delta 0), reused 0 (delta 0), pack-reused 22332
Receiving objects: 100% (22332/22332), 25.45 MiB | 24.08 MiB/s, done.
Resolving deltas: 100% (15634/15634), done.


# Read ids of testing images

In [ ]:
import json
with open('/content/dataset/test_img_ids.json') as json_file:
  ids = json.load(json_file)


# Create answer file

In [ ]:
# build the model from a config file and a checkpoint file
config_fname = 'mask_rcnn_r101_fpn_config.py'
checkpoint_file = 'mask_rcnn_2436.pth'
model = init_detector(config_fname, checkpoint_file)


In [ ]:
import os
from PIL import Image
import numpy as np
from pycocotools import mask as m

output_json = []

folder_path = '/content/dataset/test'
folder_content = os.listdir(folder_path)
for test_img in ids:
  path = os.path.join(folder_path, test_img['file_name'])
  img_id = test_img['id']
  w, h = test_img['width'], test_img['height']
  result = inference_detector(model, path)
  for i in range(len(result[0][0])):
    dicts = {}
    dicts["image_id"] = int(img_id)
    dicts["category_id"] = 1
    dicts["score"] = float(result[0][0][i][4])
    bbox = []
    # x
    bbox.append(float(result[0][0][i][0]))
    # y
    bbox.append(float(result[0][0][i][1]))
    # width
    bbox.append(float(result[0][0][i][2] - result[0][0][i][0]))
    # height
    bbox.append(float(result[0][0][i][3] - result[0][0][i][1]))
    dicts["bbox"] = bbox
    bimap = result[1][0][i]
    rle = m.encode(np.asfortranarray(bimap))
    rle['counts'] = rle['counts'].decode("utf-8")
    dicts['segmentation'] = rle
    output_json.append(dicts)


In [ ]:
json_object = json.dumps(output_json, indent=4)
with open('answer.json', 'w') as f:
  f.write(json_object)
